Import des bibliothèques python :

In [ ]:
import torch 
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import csv 
import pandas as pd
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler


In [ ]:
#1)Importation de pandas et lecture du fichier csv 
#on charge le fichier dtb.csv dans un dataframe(structure tabulaire) pandas "df"
df = pd.read_csv("dtb.csv")
#puis on affiche son contenu 
print(df)
#on affiche la dimension (nbr de lignes et de colonnes)
print(df.shape)

#2)Définir une class Dataset personnalisé 
#hérité de torch.utils.data.Dataset avec laquelle on peut créer un jeu de données 
class DtSet(Dataset):
    #on initialise l'objet
    def __init__(self, dataframe):
        #on selectionne toutes les colonnes sauf la ddernière et on recupère leurs valeurs(tableau)
        self.x = torch.tensor(dataframe.iloc[:, :-1].values, dtype=torch.float32)
        #on va sélectionner seulemenet la derr colonne qui est souvent la cible 
        self.y = torch.tensor(dataframe.iloc[:, -1].values, dtype=torch.float32)

    def __len__(self):
        return len(self.x)
    #ccette methode permet de recuperer un elem individuel à partir d'un dataset
    #elle prend un indice en entrée et renvoie la paire x,y 
    def __getitem__(self, ind):
        return self.x[ind], self.y[ind]
#creation de l'instance    
instance = DtSet(df)
#dataloader est un utilitaire pytorch qui permet de faire des batches 
#avec l'objet , la taille du lot (3 par exemple) et shuffle true permet de melanger les données
loader = torch.utils.data.DataLoader(instance, batch_size=3, shuffle=True)

print(instance.x, "\n",  instance.y)

Mauvaise Normalisation des données (Z - score) (creation of noise)

In [ ]:


"""for X, Y in loader:
    mean = X.mean()
    std = X.std()
    X = (X - mean) / std"""





In [ ]:
# True normalisation (Z - score)
#on extrait les  données  x,y,z du dataframe 
X = df[["X", "Y", "Z"]].values #(input)
#on extrait la colonne res qui est la cible dans ce cas (output)
Y = df[["res"]].values
#on crée deux normalisateurs Z-score 
#la methode z-score va transformer chaque valeur selon x_norm= x−μ/σ 
scaler_x = StandardScaler()
scaler_y = StandardScaler()
#Transformation et conversion en tensors pytorch 
"""
fit_transform : 
    fit calcule la moyenne et l'écart-type sur les données
    transform applique la normalisation Z-score
"""
X_tensor = torch.tensor(scaler_x.fit_transform(X), dtype = torch.float32)
Y_tensor = torch.tensor(scaler_y.fit_transform(Y), dtype = torch.float32)


print(X_tensor, "\n", Y_tensor)

In [ ]:
class LinRegModel(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.linear = nn.Linear(in_features, 3),
        self.linear = nn.Linear(3, 6),
        self.linear = nn.Linear(6, 6),
        self.linear = nn.Linear(6, 9),
        self.linear = nn.Linear(9, 12),
        self.linear = nn.Linear(12, 12),
        self.linear = nn.Linear(12, 9),
        self.linear = nn.Linear(9, 9),
        self.linear = nn.Linear(9, 7),
        self.linear = nn.Linear(7, 5),
        self.linear = nn.Linear(5, 3),
        self.linear = nn.Linear(3, 1)


    def forward(self, x):
        return self.linear(x)

In [ ]:
in_features = 3 
out_features = 1
# régression linéaire simple
model = LinRegModel(in_features, out_features)

In [ ]:
# Mean Squared Error comme fonction de perte
criterion = nn.MSELoss()

# Stochastic Gradient Descent (SGD) avec un learning rate de 0.1.
""" 
model.parameters() récupère tous les poids (et biais) entraînables
"""
optimizer = optim.SGD(model.parameters(), lr = 0.1)


In [ ]:
nb_epochs = 100 # nombre d’itérations

for i in range(nb_epochs):
    # Propagation avant
    """
    calcule de la sortie prédite en appliquant la régression linéaire.
    """
    outputs = model(X_tensor) 

    # fonction de perte (loss)
    # Compare les sorties prédites outputs avec les vraies valeurs Y_tensor.
    loss = criterion(outputs, Y_tensor) 

    # Remise à zéro des gradients précédents pour éviter l'accumulation des gradients d’itérations précédentes.

    optimizer.zero_grad()
    loss.backward()
    # Mise à jour des paramètres du modèle selon les gradients et la règle d’optimisation
    optimizer.step()

    print(f'Epoch [{i+ 1}], Loss: {loss.item():.9f}')



In [ ]:

new_x = np.array([[5223, 1, 2025]])

scalar_test = StandardScaler()

new_x_tensor = torch.tensor((new_x), dtype = torch.float32)

model.eval() # Évite les effets indésirables si ton modèle avait des couches comme Dropout ou BatchNorm.

with torch.no_grad(): # Évite de stocker les gradients (inutiles ici), donc plus rapide et moins de mémoire
    prediction_norm = model(new_x_tensor)

prediction_denorm = prediction_norm.item() 

print(f"Predicted for X = {new_x}: {(prediction_denorm)}")